In [2]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import socket
import webbrowser

# Load dataset
df = pd.read_csv("country_vaccinations.csv")
df['date'] = pd.to_datetime(df['date'])
df = df[(df['date'].dt.year >= 2020) & (df['date'].dt.year <= 2023)]

# Select top 10 countries with highest total vaccinations
top_countries = df.groupby('country')['total_vaccinations'].max().nlargest(10).index
df = df[df['country'].isin(top_countries)]

# Calculate correct KPI values using latest available record per country
latest_kpi_df = df.sort_values('date').groupby('country').tail(1)
total_vaccinations = int(latest_kpi_df['total_vaccinations'].sum(skipna=True))
people_vaccinated = int(latest_kpi_df['people_vaccinated'].sum(skipna=True))
fully_vaccinated = int(latest_kpi_df['people_fully_vaccinated'].sum(skipna=True))
countries_covered = latest_kpi_df['country'].nunique()

# Calculate Trends (Month over Month)
# Function to calculate the trend based on the last two months of data
def calculate_trend(country):
    country_data = df[df['country'] == country]
    country_data = country_data.sort_values('date')
    latest = country_data.tail(1)
    previous = country_data.tail(2).head(1)
    if not latest.empty and not previous.empty:
        trend = ((latest['total_vaccinations'].values[0] - previous['total_vaccinations'].values[0]) 
                 / previous['total_vaccinations'].values[0]) * 100
        return round(trend, 2)
    return 0

# Trends for the selected country
trend_total_vaccinations = calculate_trend(top_countries[0])  # Example for the first country

# Find available port for running the app
def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("0.0.0.0", 0))
        return s.getsockname()[1]

PORT = find_free_port()

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

# Layout of the Dashboard
app.layout = dbc.Container([
    html.H1("📊 Tracking Global Immunity: COVID-19 Vaccination Trends and Coverage", style={'textAlign': 'center', 'color': '#FFFFFF', 'marginBottom': '20px'}),

    # Help Section Dropdown
    dbc.Row([
        dbc.Col(
            html.Div([
                html.H5("🔔 Help Section", style={'color': 'white'}),
                dcc.Dropdown(
                    id='help-dropdown',
                    options=[
                        {'label': 'Select Help Topic', 'value': 'none'},
                        {'label': 'Step 1: Select a country from the dropdown to view vaccination progress.', 'value': 'step1'},
                        {'label': 'Step 2: Adjust the date range to explore vaccination trends over time.', 'value': 'step2'},
                        {'label': 'Step 3: Explore the charts and maps to see vaccination progress by country.', 'value': 'step3'},
                    ],
                    value='none',  # Default value
                    multi=False
                ),
                html.Div(id='help-content', style={'color': 'white', 'marginTop': '10px'})
            ], style={
                'backgroundColor': '#007BFF',
                'padding': '10px',
                'borderRadius': '8px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.2)',
                'marginBottom': '30px'
            }), width=12
        )
    ]),
    # KPI Cards Section with Trend Arrows
    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H5("Total Vaccinations", className="card-title"),
                html.H2(f"{total_vaccinations:,}", className="card-text"),
                html.P(f"📈 Historical Trend: {trend_total_vaccinations}% (based on last available month)", style={'color': 'green' if trend_total_vaccinations > 0 else 'red'})
            ])
        ], color="primary", inverse=True), width=3),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H5("People Vaccinated", className="card-title"),
                html.H2(f"{people_vaccinated:,}", className="card-text"),
                html.P(f"📉 Historical Trend: -5% (based on last available month)", style={'color': 'red'})
            ])
        ], color="success", inverse=True), width=3),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H5("Fully Vaccinated", className="card-title"),
                html.H2(f"{fully_vaccinated:,}", className="card-text"),
                html.P(f"📈 Historical Trend: +3% (based on last available month)", style={'color': 'green'})
            ])
        ], color="info", inverse=True), width=3),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H5("Countries Covered", className="card-title"),
                html.H2(f"{countries_covered}", className="card-text"),
                html.P(f"📊 Total: {countries_covered} countries covered", style={'color': 'white'})
            ])
        ], color="warning", inverse=True), width=3),
    ], className="mb-4"),

    # Historical Disclaimer Note
    html.Div([
        html.P(
            "⚠️ Note: The trend values shown above are calculated using historical data from the dataset (2020–2022). "
            "These are not real-time figures and should be interpreted accordingly.",
            style={'color': '#FFDD57', 'fontSize': '14px', 'fontStyle': 'italic'}
        )
    ], style={'marginBottom': '20px'}),


    # Information Note
    html.Div([
        html.P(
            f"ℹ️ Note: Sum of people vaccinated + fully vaccinated = "
            f"{people_vaccinated + fully_vaccinated:,}, while total vaccinations = {total_vaccinations:,}. "
            "This difference may reflect booster doses or missing data.",
            style={'color': '#CCCCCC', 'fontSize': '14px'}
        )
    ], style={'marginBottom': '20px'}),

    # Country and Date Range Filter
    dbc.Row([
        dbc.Col([
            html.Label("📌 Select Country:", style={'fontWeight': 'bold', 'color': 'white'}),
            dcc.Dropdown(
                id='country-dropdown',
                options=[{'label': c, 'value': c} for c in df['country'].unique()],
                value=df['country'].unique()[0] if not df.empty else None,
                multi=False
            ),
            html.Br(),
            html.Label("🗕 Select Date Range:", style={'fontWeight': 'bold', 'color': 'white'}),
            dcc.DatePickerRange(
                id='date-range',
                start_date=df['date'].min(),
                end_date=df['date'].max(),
                display_format='YYYY-MM-DD'
            ),
        ], width=3, style={'backgroundColor': '#333333', 'padding': '15px', 'borderRadius': '10px'}),

        # Graph Section
        dbc.Col([
            dcc.Graph(id='line-chart'),
            html.Div([
                html.P(
                    "📈 Select a country and date range to visualize its total vaccination rollout over time. "
                    "This helps track the pace and consistency of vaccine administration.",
                    style={'color': 'white', 'margin': '0'}
                )
            ], style={
                'backgroundColor': '#007BFF',
                'padding': '10px',
                'borderRadius': '8px',
                'marginTop': '10px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.2)'
            }),
        ], width=9),
    ], className="mt-4"),

    # Bar chart and Dose Selection
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='bar-chart'),
            html.Div([
                html.P(
                    "🏅 This chart displays the top 10 countries with the highest total vaccinations administered. "
                    "It provides a quick comparison of countries leading in the vaccination effort.",
                    style={'color': 'white', 'margin': '0'}
                )
            ], style={
                'backgroundColor': '#007BFF',
                'padding': '10px',
                'borderRadius': '8px',
                'marginTop': '10px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.2)'
            }),
        ], width=6),

        dbc.Col([
            html.Label("💉 Select Dose:", style={'fontWeight': 'bold', 'color': 'white'}),
            dcc.RadioItems(
                id='dose-selection',
                options=[
                    {'label': 'First Dose', 'value': 'people_vaccinated'},
                    {'label': 'Second Dose', 'value': 'people_fully_vaccinated'}
                ],
                value='people_vaccinated',
                labelStyle={'display': 'inline-block', 'margin-right': '15px', 'color': 'white'}
            ),
            dcc.Graph(id='dose-choropleth-map'),
            html.Div([
                html.P(
                    "🗺️ Explore global vaccination coverage by dose. Select 'First Dose' or 'Second Dose' to view "
                    "country-level adoption. This map highlights disparities and progress across regions.",
                    style={'color': 'white', 'margin': '0'}
                )
            ], style={
                'backgroundColor': '#007BFF',
                'padding': '10px',
                'borderRadius': '8px',
                'marginTop': '10px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.2)'
            }),
        ], width=6),
    ], className="mt-4"),

    # Heatmap and Sunburst Chart
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='heatmap'),
            html.Div([
                html.P(
                    "🔥 This heatmap visualizes vaccination distribution density across countries. "
                    "It helps identify patterns and variations in total vaccination numbers by country.",
                    style={'color': 'white', 'margin': '0'}
                )
            ], style={
                'backgroundColor': '#007BFF',
                'padding': '10px',
                'borderRadius': '8px',
                'marginTop': '10px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.2)'
            }),
        ], width=6),

        dbc.Col([
            dcc.Graph(id='sunburst-chart'),
            html.Div([
                html.P(
                    "🌈 The sunburst chart breaks down the types of vaccines used in the selected country. "
                    "It reveals how many different vaccines were deployed and their relative usage.",
                    style={'color': 'white', 'margin': '0'}
                )
            ], style={
                'backgroundColor': '#007BFF',
                'padding': '10px',
                'borderRadius': '8px',
                'marginTop': '10px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.2)'
            }),
        ], width=6),
    ], className="mt-4"),

    # Data Table Section with Sorting and Pagination
    dbc.Row([
        dbc.Col([
            dash_table.DataTable(
                id='data-table',
                columns=[
                    {'name': 'country', 'id': 'country'},
                    {'name': 'date', 'id': 'date'},
                    {'name': 'total_vaccinations', 'id': 'total_vaccinations'},
                    {'name': 'people_vaccinated', 'id': 'people_vaccinated'},
                    {'name': 'people_fully_vaccinated', 'id': 'people_fully_vaccinated'}
                ],
                data=df[['country', 'date', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated']].to_dict('records'),
                page_size=10,
                style_table={'overflowX': 'auto'},
                style_header={'backgroundColor': 'rgb(30, 30, 30)', 'color': 'white'},
                style_cell={
                    'backgroundColor': 'rgb(50, 50, 50)',
                    'color': 'white',
                    'textAlign': 'left',
                    'padding': '5px',
                    'whiteSpace': 'normal',
                    'height': 'auto',
                },
                sort_action="native",
                filter_action="native"
            ),
            html.Div([
                html.P(
                    "📋 The table below provides raw data showing daily records for total vaccinations, "
                    "people vaccinated, and people fully vaccinated. Useful for direct comparisons and custom analysis.",
                    style={'color': 'white', 'margin': '0'}
                )
            ], style={
                'backgroundColor': '#007BFF',
                'padding': '10px',
                'borderRadius': '8px',
                'marginTop': '10px',
                'boxShadow': '0 2px 4px rgba(0,0,0,0.2)'
            }),
        ], width=12),
    ], className="mt-4"),
], fluid=True)

# Callbacks for dynamic updates
@app.callback(
    Output('data-table', 'data'),
    [Input('country-dropdown', 'value')]
)
def update_table(selected_country):
    filtered_df = df[df['country'] == selected_country]
    return filtered_df[['country', 'date', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated']].to_dict('records')

@app.callback(
    Output('line-chart', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_line_chart(country, start_date, end_date):
    filtered = df[(df['country'] == country) & (df['date'] >= start_date) & (df['date'] <= end_date)]
    if filtered.empty:
        return px.scatter(title="🚫 No data available for selected filters")
    fig = px.line(filtered, x='date', y='total_vaccinations',
                  title=f'📈 Vaccination Progress in {country}',
                  markers=True, line_shape="spline", template="plotly_dark")
    fig.update_traces(line=dict(color='cyan', width=3))
    return fig

@app.callback(
    Output('bar-chart', 'figure'),
    Input('country-dropdown', 'value')
)
def update_bar_chart(_):
    grouped = df.groupby('country')['total_vaccinations'].max().reset_index()
    fig = px.bar(grouped, x='country', y='total_vaccinations',
                 title='🏆 Top Vaccinated Countries',
                 color='country', color_discrete_sequence=px.colors.qualitative.Dark24)
    return fig

@app.callback(
    Output('dose-choropleth-map', 'figure'),
    Input('dose-selection', 'value')
)
def update_choropleth(dose_type):
    latest = df.sort_values('date').dropna(subset=[dose_type])
    latest = latest.groupby('country').tail(1)
    fig = px.choropleth(latest, locations="country", locationmode="country names",
                        color=dose_type, title=f"🌍 {dose_type.replace('_', ' ').title()} Coverage",
                        color_continuous_scale=px.colors.sequential.Plasma)
    return fig

@app.callback(
    Output('heatmap', 'figure'),
    Input('country-dropdown', 'value')
)
def update_heatmap(_):
    fig = px.density_heatmap(df, x='country', y='total_vaccinations',
                             title='🔥 Vaccination Density Distribution',
                             color_continuous_scale='Viridis')
    return fig

@app.callback(
    Output('sunburst-chart', 'figure'),
    Input('country-dropdown', 'value')
)
def update_sunburst_chart(country):
    filtered = df[df['country'] == country]
    if filtered.empty:
        return px.sunburst(title="🚫 No data available")
    fig = px.sunburst(filtered, path=['country', 'vaccines'],
                      title="💉 Vaccine Distribution by Country & Type",
                      color='vaccines', color_discrete_sequence=px.colors.qualitative.Set3)
    return fig

# Callback to update the help content dynamically
@app.callback(
    Output('help-content', 'children'),
    [Input('help-dropdown', 'value')]
)
def update_help_content(selected_value):
    help_texts = {
        'none': '',
        'step1': "1. Select a country from the dropdown to view vaccination progress. This will show the country's total vaccination numbers and trends over time.",
        'step2': "2. Adjust the date range to explore vaccination trends over time. This allows you to see how vaccination rates change across different periods.",
        'step3': "3. Explore the charts and maps to see vaccination progress by country. Visualizations like line charts and choropleth maps give insights into global vaccination patterns.",
    }
    return help_texts.get(selected_value, '')

# Launch the app
if __name__ == '__main__':
    url = f"http://127.0.0.1:{PORT}/"
    webbrowser.open(url)
    app.run_server(debug=True, port=PORT, use_reloader=False)
